# 18.06 pset 4 solutions

## Problem 1

Suppose that you have the $PA=LU$ factorization of an invertible $m\times m$ matrix $A$.   Explain how you can solve $A^T x = b$ in $\sim m^2$ operations (i.e., without re-doing elimination on $A^T$).

In Julia, the LU factorization is computed with `LU = lufact(A)`.  The transpose of a real matrix is `A'`.  You can solve $A^T x = b$ with either `A' \ b` (which re-does elimination) or with `LU' \ b` (which re-uses the factorization).  By trying this for a few sizes, check whether the times indeed scale roughly with $\sim m^3$ or $\sim m^2$ as expected:

### Solution

Remember that for any matrices $X, Y$ where the multiplication $XY$ makes sense, we have $(XY)^T = Y^TX^T$.  In particular, taking the transpose of both sides of the equation $PA = LU$ gives $A^TP^T = U^TL^T$, or $A^T = U^TL^T(P^T)^{-1}$.  RSo, the equation $A^Tx = b$ with unknown $x$ is the same as the equation $U^TL^T(P^T)^{-1}x = b$.  But $U^T$ is *lower*-triangular, $L^T$ is *upper*-triangular, so we can solve this quickly by parenthesizing it as $U^T(L^T((P^T)^{-1}x)) = b$ and solving from the outside in:

1. Solve $U^T z = b$ for $z$ by forward-substitution ($\sim m^2$)
2. Solve $L^T y = z$ for $y$ by back-substitution ($\sim m^2$)
3. Solve $(P^T)^{-1}x = y$ by $x = P^T y$.  This is just a matrix-vector permutation, so it can be done in $\sim m^2$ operations.  But if we are a bit more clever, we realize that it is a permutation (or, more simply, $P^T$ only has one nonzero element per row: it is *sparse*), so we can actually do it in $\sim m$ operations.

Hence the whole thing requires $\sim m^2$ operations.

* **Remark**: Some of you wondered whether we should count the cost of "computing" a transpose like $U^T$.  This doesn't change the result: at worst, copying an $m\times m$ matrix to its transpose involves $\sim m^2$ operations (copying $m^2$ elements to their new locations in memory).   (In practice, computer software would often avoid explicitly transposing the matrix in memory, since working with the transposed matrix just involves accessing the same entries in a different order.)

When we run the provided code, we see that indeed solving the equation in this manner takes $\sim m^2$ time once the $LU$-decomposition is known, while solving it directly takes $\sim m^3$ time.  If it were exactly $m^2$ and $m^3$, doubling the size would multiply the time by 4 and 8, respectively,   Instead we get ratios of 7.1 and 4.7, respectively, which is pretty close.  (And in absolute terms, solving with `LU' \ b` is more than 100x faster than `A' \ b` for both of these sizes.)

(Your exact numbers will vary, of course.  But you should still see a stark difference in the timings and the growth between solving $A^Tx=b$ from scratch or re-using an existing LU factorization.)

In [2]:
BLAS.set_num_threads(1) # disable multi-threading for benchmarking
for m in (2000, 4000) # try for a couple sizes
    A = rand(m,m)
    b = rand(m)
    LU = lufact(A) # an object that stores the LU factorization of A efficiently
    
    # benchmark, taking the minimum of three times to reduce timing noise
    t1 = min(@elapsed(A' \ b), @elapsed(A' \ b), @elapsed(A' \ b))
    t2 = min(@elapsed(LU'\ b), @elapsed(LU'\ b), @elapsed(LU'\ b))
    println("m = $m, time for A' \\ b = $t1 seconds, time for LU \\ b = $t2 seconds")
end

m = 2000, time for A' \ b = 0.371909352 seconds, time for LU \ b = 0.002744389 seconds
m = 4000, time for A' \ b = 2.642134642 seconds, time for LU \ b = 0.012963979 seconds


In [3]:
2.642134642 / 0.371909352

7.10424362224696

In [4]:
0.012963979 / 0.002744389

4.723812477021298

## Problem 2

(Based on Strang, section 4.1, problem 9.)

The following is an important property of the very important matrix $A^T A$ (for real matrices) that will come up several times in 18.06:

* If $A^TAx=0$ then $Ax=0$.  Reason: If $A^TAx=0$, then $Ax$ is in the nullspace of $A^T$ and also in the ?????? of $A$, and those spaces are ???????.  Conclusion: $N(A^T A) = N(A)$.

* Alternative proof: $A^TAx=0$, then $x^T A^T Ax = 0 = (Ax)^T (Ax)$.  Why does this imply that $Ax=0$?   (Hint: if $y^Ty = 0$, can we have $y\ne 0$?)

* If $A$ is a random $m \times n$ matrix, what can you conclude about the ranks of $A^T A$ and $A A^T$?   Try it in Julia for a $5 \times 7$ random matrix:

### Solution

* If $A^TAx=0$ then $Ax=0$.  Reason: If $A^TAx=0$, then $Ax$ is in the nullspace of $A^T$ and also in the **column space** of $A$, and those spaces are **orthogonal** (so $Ax$ is orthogonal to itself, and therefore is 0). Conclusion: $N(A^T A) = N(A)$.

* Alternative proof: $A^TAx=0$, then $x^T A^T Ax = 0 = (Ax)^T (Ax)$.  The only vector $y$ that satisfies $y^Ty = 0$ is the zero vector; indeed, if $y = (y_1, ..., y_n)^T$, then $y^TY = y_1^2 + \cdots + y_n^2$.  The only way a sum of squares of real numbers can be zero is if each of the numbers in the sum was already 0, i.e. $y_1 = \cdots = y_n = 0$, i.e. $y = 0$.  Since this applies for $y = Ax$, we have $Ax = 0$.

* If $A$ is a random $m \times n$ matrix, what can you conclude about the ranks of $A^T A$ and $A A^T$?   Well, a random matrix has full rank.  So, if $A$ is a random $m \times n$ matrix, then it has rank $\min(n, m)$, and the same is true for $A^T$.  At the same time, by the first bullet point above we have that $N(A) = N(A^TA)$, so $A$ and $A^TA$ have the same nullity; but they also have the same number of columns, so by the rank-nullity theorem (the relationship between the rank and the dimension of the nullspace) they have the same rank!  Replacing $A$ with $A^T$ from the start, the same holds for $A^T$ and $AA^T$.  In particular, all of the matrices $A, A^T, A^TA, AA^T$ have the same rank $\min(m, n)$ when we choose $A$ randomly.

Running the suggested code, we get rank $5 = \min(5, 7)$ each time, as prediced.

In [3]:
A = randn(5,7)
rank(A'*A)

5

In [4]:
rank(A*A')

5

## Problem 3

Suppose we *already know* the inverse $A^{-1}$ of an $m \times m$ matrix $A$.   Now, we want to find the inverse $(A + uv^T)^{-1}$, where $u$ and $v$ are $m$-component column vectors.   Ideally, we'd like to do this without re-doing the whole matrix-inversion process!

* Remark: $A + uv^T$ is called a "rank-1 update" of $A$ because you are adding the rank-1 matrix $uv^T$.

Find the scalar $\alpha$ such that:

$$
(A + uv^T)^{-1} = A^{-1} - \frac{A^{-1} uv^T A^{-1}}{\alpha}
$$

(Multiply the right-hand side by $A + uv^T$ and try to get $I$.)

Check your answer for a random $4\times 4$ matrix $A$ and random vectors $u$ and $v$, by replacing the `????` with your answer:

### Solution

As suggested, treat the constant $\alpha$ as unknown and optimistically multiply the proposed formula on the right by $A + uv^T$:

$$\left(A^{-1} - \frac{A^{-1} uv^T A^{-1}}{\alpha}\right)(A + uv^T) = I + A^{-1}uv^T - \frac{A^{-1} uv^T + A^{-1}uv^TA^{-1}uv^T}{\alpha}$$ $$= I + A^{-1}uv^T - \frac{A^{-1}u(1 + v^TA^{-1}u)v^T}{\alpha}$$ $$= I + \left(1 - \frac{1 + v^TA^{-1}u}{\alpha}\right)A^{-1}uv^T$$ (notice that $v^TA^{-1}u$ is a scalar!  So, the above expression equals $I$ if and only if $$1 - \frac{1 + v^TA^{-1}u}{\alpha} = 0,$$ i.e. if and only if $\boxed{\alpha = 1 + v^TA^{-1}u}$.

Conclusion:  If $A^{-1}$ is already known and if the scalar $\alpha = 1 + v^TA^{-1}u$ *isn't zero*, then $A + uv^T$ is invertible with inverse given by the formula in the problem statement for this value of $\alpha$.

* **Remark**: This result is actually the famous and important [Sherman–Morrison formula](https://en.wikipedia.org/wiki/Sherman%E2%80%93Morrison_formula).

This is verified in a particular case by running the provided code and entering our definition of α:

In [1]:
A = rand(4,4)
u = rand(4)
v = rand(4)

A⁻¹ = inv(A)

α = 1 + v'*inv(A)*u

1.4023041680400459

In [2]:
# check that the difference is a matrix with tiny entries (just roundoff errors):
inv(A + u*v') - (A⁻¹ - (A⁻¹*u)*(v'*A⁻¹)/α)

4×4 Array{Float64,2}:
 -1.66533e-16   5.55112e-16  -6.66134e-16   0.0        
  2.22045e-16  -2.22045e-16  -4.44089e-16   3.88578e-16
  2.77556e-16  -4.996e-16     2.22045e-16   2.22045e-16
  0.0           6.66134e-16   0.0          -4.996e-16  

## Problem 4

(Based on Strang, section 2.7, problem 16.)

A *symmetric* matrix $S$ is a matrix that equals its transpose: $S = S^T$.   Suppose that $A=A^T$ and $B=B^T$ are two symmetric matrices, and $C$ is some other matrix (possibly nonsymmetric), all of the same size $m\times m$.  Which of the following are certainly symmetric?

* $A^2 - B^2$
* $(A+B)(A-B)$ (note that this does *not* generally equal $A^2 - B^2$ ... why?)
* $C^T A C$
* $A^T C A$
* $ABA$
* $C + C^T$
* $C - C^T$
* $ABAB$

You can try these with some random matrices to check your answers, but your solutions should explain your answers with equations.

In [ ]:
A = rand(-5:5, 4,4); A = A + A'  # random
B = rand(-5:5, 4,4); B = B + B'  #   symmetric 4×4 matrices
C = rand(-5:5, 4,4)  # random (probably) non-symmetric 4×4 matrix

A^2 - B^2

### Solution

The idea is to just take the transpose in each case and see what you get.  If you get the same thing, the matrix in question was symmetric.  If you don't get the same form, you can usually see why, and you can then cook up some choices of the matrices that really show the result doesn't have to be symmetric.

The calculations below use repeatedly the facts that $(X + Y)^T = X^T + Y^T$ and $(XY)^T = Y^TX^T$.  We also of course use the facts that $A^T = A$ and $B^T = B$ each time.

* $A^2 - B^2$: MUST be symmetric. Taking transposes, we get $$(A^2 - B^2)^T = (AA)^T - (BA)^T = A^TA^T - B^TB^T = A^2 - B^2$$
* $(A+B)(A-B)$: DOES NOT need to be symmetric.  First, multiplying out the expression we get $(A^2 - B^2) + (BA - AB)$.  This *does not have to equal* $A^2 - B^2$ because it's possible that $AB \neq BA$ (i.e. when $A$ and $B$ don't *commute*.  Taking the transpose gives: $$(A^2 - B^2)^T + (BA - AB)^T = A^2 - B^2 - A^TB^T - B^TAT^ = A^2 - B^2 - (BA - AB).$$  So, we see that $(A + B)(A - B)$ is symmetric if and only if $BA - AB = AB - BA$, i.e. if and only if $AB = BA$, i.e when $A$ and $B$ commute.  We can find symmetric matrices $A$ and $B$ that don't commute pretty easily, for example using our knowledge of row/column operations.  Remember that row operations are achieved by left multiplications, and column operations by right multiplications.  So, if we take $A$ to be the symmetric permutation matrix $$A = \begin{pmatrix} 0 & 1 \\ 1 & 0\end{pmatrix}$$ then $AB$ will be $B$ with its rows swapped, and $BA$ will be $B$ with its columns swapped.  So, choosing $B$ to be a symmetric matrix that looks different if we swap its rows or columns, for example $$B = \begin{pmatrix} 1 & 0 \\ 0 & 2\end{pmatrix}$$ we get a concrete counterexample.

  - **Remark** The matrix $AB - BA$ is called the [*commutator*](https://en.wikipedia.org/wiki/Commutator) of $A$ and $B$, and measures in a sense how/if $A$ and $B$ commute.  It's an extremely important construction that appears all over the place, for example in making precise the [Heisenberg uncertainty principle](https://en.wikipedia.org/wiki/Uncertainty_principle)

* $C^T A C$ MUST be symmetric.  Taking transposes, we get: $$(C^TAC)^T = C^TA^TC = C^TAC$$ which is what we started with.
* $A^T C A$ DOES NOT need to be symmetric.  Taking transposes we get $$(A^TCA)^T = A^TC^TA$$ but we don't have to have that $C = C^T$. In fact, if $A$ is invertible, then so is $A^{-1}$, so we can "cancel" the $A^T$ on the left and the $A$ on the right when comparing $(A^TCA)^T = A^TC^TA$ and $A^TCA$, obtaining $C = C^T$, so $A^TCA$ is symmetric if and only if $C$ is symmetric (again, when $A$ is invertible).  So choosing $A = I$ and $C$ anything non-symmetric will produce a non-symmetric matrix $A^TCA$.
* $ABA$ MUST be symmetric.  Taking transposes, we get $$(ABA)^T = A^TB^TA^T = ABA,$$ the same as we started with.  More generally, any [palindrome](https://en.wikipedia.org/wiki/Palindrome) product of symmetric matrices will still be symmetric.
* $C + C^T$ MUST be symmetric.  Taking transposes, we get $$(C + C^T) = C^T + C = C + C^T,$$ the same as we started with.
* $C - C^T$ DOES NOT need to be symmetric.  Taking transposes, we get $$(C - C^T)^T = C^T - C = -(C - C^T),$$ the *negative* of what we started with.  The only way a matrix can be equal to its negative is if it was already $0$; so the only way $C - C^T$ can be symmetric is if it was zero already, i.e. if $C = C^T$, i.e. if $C$ was symmetric.  So any non-symmetric $C$ will give a counterexample.

  - **Remark** A matrix $X$ that satisfies $X^T = -X$ is called *antisymmetric*.  It's a basic fact that any matrix can be written uniquely as a sum of a symmetric matrix with an antisymmetric matrix.  Indeed, if $X$ is any square matrix, we can write $$X = \frac{X + X^T}{2} + \frac{X - X^T}{2}$$ and the first term is symmetric (by two bullet points up) and the second term is anti-symmetric (by one bullet point up).  The uniqueness can be seen like this.  Suppose $X = Y + Z = Y' + Z'$ where $Y, Y'$ are symmetric and $Z, Z'$ are antisymmetric.  Then $Y - Y' = Z - Z'$.  But as $Y, Y'$ are symmetric, so is $Y - Y'$.  As $Z, Z'$ are antisymmetric, so is $Z - Z'$.  But $Y - Y'$ and $Z - Z'$ are equal, and the only matrix that is simultaneously symmetric and antisymmetric is the 0 matrix; so $Y - Y' = Z - Z' = 0$, so $Y = Y$ and $Z = Z'$ and the decompositions $X = Y + Z = Y' + Z'$ were the same to begin with!  As a final note, if we think of a $m \times m$ matrix as a vector of length $m^2$, then any symmetric matrix is orthogonal to any antisymmetric matrix (why?), so the above actually shows that the *subspaces* of symmetric and anti-symmetric matrices are orthogonal complements of one another.

* $ABAB$ DOES NOT need to be symmetric.  Taking transposes, we get $$(ABAB)^T = B^TA^TB^TA^T = BABA$$ so for $ABAB$ to by symmetric means that $ABAB = BABA$.  We can cook up a counterexample using a similar strategy to what we did in the $(A - B)(A + B)$ problem.  In particular, let's just choose some random matrices $A, B$ that are symmetric; they probably won't commute very well, and so $ABAB$ and $BABA$ should be different.  Indeed, this is what the following calculation in Julia shows (we don't get 0):

In [3]:
A = rand(-5:5, 4,4); A = A + A'  # random
B = rand(-5:5, 4,4); B = B + B'  #   symmetric 4×4 matrices
C = rand(-5:5, 4,4)  # random (probably) non-symmetric 4×4 matrix

A*B*A*B - B*A*B*A

4×4 Array{Int64,2}:
  1230  -118  -2540    2696
  2415   339  -4148    4316
 -6333  1126  14701  -15640
  4364  -605  -9303    9916

## Problem 5

(From Strang, seiton 3.5, problem 3.)

Find a basis for each of the four subspaces associated with $A$, given its upper-triangular form and elimination steps:

$$
A = \begin{pmatrix} 0 & 1 & 2 & 3 & 4 \\ 0 & 1 & 2 & 4 & 6 \\ 0 & 0 & 0 & 1 & 2 \end{pmatrix} =
\begin{pmatrix} 1 & 0 & 0 \\ 1 & 1 & 0 \\ 0 & 1 & 1 \end{pmatrix}
\begin{pmatrix} 0 & 1 & 2 & 3 & 4 \\ 0 & 0 & 0 & 1 & 2 \\ 0 & 0 & 0 & 0 & 0 \end{pmatrix}
$$

### Solutions

Write the given matrix equation as $$A = EB$$ just to give the two matrices on the right names.

Remember that row opertions preserve the nullspace and the rowspace, but mess up the column space and left nullspace.  So we'll start with the spaces involving the rows.

* $C(A^T)$: The nonzero rows (transposed) of $B$ give a basis of the row space $$C(A^T) = C(B^T): C(A^T) = \mbox{Span }\left\{\begin{pmatrix}0\\1\\2\\3\\4\end{pmatrix}, \begin{pmatrix}0\\0\\0\\1\\2\end{pmatrix}\right\}$$

* $N(A)$: There are three free (=non-pivot) columns in $B$ and $N(A) = N(B)$, so $N(A)$ has dimension 3.  The free variables are $x_1, x_3, x_5$.  Setting one of them to 1 and the others to zero and solving gives the special solutions (= basis for $N(A)$) (remember, what we're looking for here is how to produce one of the free columns as a linear combination of the pivot columns, which is easy just by eye-balling it even though it's not quite in $RREF$ form): $$N(A) = \text{Span}\left\{\begin{pmatrix}1\\0\\0\\0\\0\end{pmatrix}, \begin{pmatrix}0\\-2\\1\\0\\0\end{pmatrix}, \begin{pmatrix}0\\2\\0\\-2\\1\end{pmatrix}\right\}$$

* To get a basis for $C(A)$, we just need to take the columns of $A$ occuring in the same positions as the pivot columns of $B$.  So we get $$C(A) = \text{Span}\left\{\begin{pmatrix}1\\1\\0\end{pmatrix}, \begin{pmatrix}3\\4\\1\end{pmatrix}\right\}$$

* To get a basis for the (1-dimensional) left nullspace $N(A^T)$, there are several approaches:

  - The vector $e_3 = \begin{pmatrix} 0 \\ 0 \\ 1 \end{pmatrix}$ is a basis for $N(B^T)$, since that picks out the zero column of $B^T$.  This is *not* in $N(A^T)$ however.  As explained in class, we need to transform via $A=EB \implies B^T = A^T (E^T)^{-1}$, which gives the equation $0 = B^T e_3 = A^T (E^T)^{-1} e_3$, so the vector $x = (E^T)^{-1} e_3$ is in $N(A^T)$ and we can find $x$ by solving $E^T x = e_3$ by backsubstitution ($E^T$ is upper-triangular), i.e. solve $\begin{pmatrix} 1 & 1 & 0 \\ 0 & 1 & 1 \\ 0 & 0 & 1 \end{pmatrix} x = \begin{pmatrix} 0 \\ 0 \\ 1\end{pmatrix}$.  This yields $x = \begin{pmatrix}1\\-1\\1\end{pmatrix}$ as our basis vector.
  
  - We could just re-do elimination on $A^T$ and get its nullspace the "usual" way from the free column.  In theory, it's a little wasted effort, but you're pretty good at elimination by now and $A$ is small, so this might actually be the least error-prone of the methods.

  - We can use our knowledge of $C(A)$; we're just looking for it's orthogonal complement.  The orthogonal complement of a plane in $\mathbb{R}^3$ is just a line, so we're just looking for a single nonzero vector in the orthogonal complement of $C(A)$.  A vector $(x_1, x_2, x_3)^T$ is such a vector if and only if $x_1 + x_2 = 0$ and $3x_1 + 4x_2 + x_3 = 0$.  So we need $x_2 = -x_1$, and substituting into the second equation we get $3x_1 - 4x_2 + x_3 = 0$, i.e. $x_3 = x_2$.  So the vector $(1, -1, 1)^T$ works, and we have $$N(A^T) = \text{Span}\left\{\begin{pmatrix}1\\-1\\1\end{pmatrix}\right\}.$$
  
  - 18.02 would tell you that we could have also used the cross product of the $C(A)$ basis vectors to get an orthogonal vector, which gives a basis for $N(A^T)$.

# Problem 6

(From Strang, section 3.5, problem 4)

Construct matrices with each of the following properties, or explain why it is impossible:

* Column space contains $\begin{pmatrix} 1\\1\\0 \end{pmatrix}$, $\begin{pmatrix} 0\\0\\1 \end{pmatrix}$, and row space contains $\begin{pmatrix} 1\\2 \end{pmatrix}$, $\begin{pmatrix} 2 \\5 \end{pmatrix}$

* Column space has basis $\begin{pmatrix} 1\\1\\3 \end{pmatrix}$, nullspace has basis $\begin{pmatrix} 3\\1\\1 \end{pmatrix}$

* Dimension of nullspace = 1 + dimension of left nullspace

* Nullspace contains $\begin{pmatrix} 1\\3 \end{pmatrix}$, column space contains $\begin{pmatrix} 3\\1 \end{pmatrix}$

* Row space = column space, nullspace ≠ left nullspace.

### Solutions

* Column space contains $\begin{pmatrix} 1\\1\\0 \end{pmatrix}$, $\begin{pmatrix} 0\\0\\1 \end{pmatrix}$, and row space contains $\begin{pmatrix} 1\\2 \end{pmatrix}$, $\begin{pmatrix} 2 \\5 \end{pmatrix}$

  - **Solution:** It CAN be done.  The two vectors given in the column space are visibly linearly independent, as are the two vectors in the row space, so the dimensions of these would need to be the same - which is necessary for any matrix!  We can also see that the columns have height 3 and the rows have height 2, so the matrix needs to be $3 \times 2$.  Also, the given row vectors span all of $\mathbb{R}^2$, so as soon as we get the column space right the row space will also be right ($\mathbb{R}^2$ is the only subspace of $\mathbb{R}^2$ containing the given vectors).  Using the given columns, we can then just write down the answer:

$$\begin{pmatrix} 1 & 0 \\ 1 & 0 \\ 0 & 1\end{pmatrix}$$

* Column space has basis $\begin{pmatrix} 1\\1\\3 \end{pmatrix}$, nullspace has basis $\begin{pmatrix} 3\\1\\1 \end{pmatrix}$

  - **Solution:** This is NOT possible.  We can see that the matrix would need to be $3 \times 3$.  But if the column space and nullspace each have a basis of size 1, then they each have dimension 1, and as $1 + 1 = 2 \neq 3$ this violates the rank-nullity theorem.

* Dimension of nullspace = 1 + dimension of left nullspace

  - **Solution:** This CAN be done.  Let's first think about the size of such a matrix.  Say it's $m \times n$ and has rank $r$.  Then if $d$ is the dimension of the nullspace, so that $d - 1$ is the dimension of the left nullspace, by the rank-nullity theorem we have $d + r = n$ and $(d - 1) + r = m$, so we need to have that $n = m + 1$, i.e. there should be one more column than row.  Any random matrix of this form without additional structure will do the trick, for example $$\begin{pmatrix} 1 & 2\end{pmatrix}$$

* Nullspace contains $\begin{pmatrix} 1\\3 \end{pmatrix}$, column space contains $\begin{pmatrix} 3\\1 \end{pmatrix}$

  - **Solution:** This CAN be done.  Notice that the conditons mean that each of the nullspace and the column space are nonzero.  They also live in $\mathbb{R}^2$, so they have to either be lines or all of $\mathbb{R}^2$.  But as their dimensions need to add to 2 by rank-nullity, they must both be lines.  So in fact the given vectors form *bases* for the nullspace and column space.  Since there is only one vector in a basis for the column space, the two columns have to be scalar multiples of $(3, 1)^T$.  That $(1, 3)^T$ is in the nullspace means that the second column needs to be -3 times the first column.  So the following matrix does the trick: $$\begin{pmatrix} 3 & -9 \\ 1 & -3\end{pmatrix}$$

* Row space = column space, nullspace ≠ left nullspace.

  - **Solution:** This is IMPOSSIBLE.  The nullspace can be characterized as the orthogonal complement of the row space, and the left nullspace can be characterized as the orthogonal complement of the column space.  But if the row space and the column space are *equal*, then so are their orthogonal complements, so the nullspace and left nullspace would need to coincide as well.

## Problem 7

(From Strang, section 4.1, problems 14–15.)

* For $A = \begin{pmatrix}1 & 2\\1 & 3 \\ 1& 2\end{pmatrix}$ and $B = \begin{pmatrix}5 & 4 \\ 6 & 3 \\ 5 & 1 \end{pmatrix}$, the column spaces $C(A)$ and $C(B)$ are planes in $\mathbb{R}^3$.  Two planes cannot be orthogonal because they always intersect in a common line.   Find a (nonzero) vector in both $C(A)$ and $C(B)$ by considering the ________ space of the matrix $\begin{pmatrix} A & B \end{pmatrix}$.

* Extend this tor a $p$-dimensional subspace $V$ and a $q$-dimensional subspace $p$ of $\mathbb{R}^n$.  What inequality on $p+q$ guarantees that $V$ intersects $W$ in a nonzero vector (and hence $V$ and $W$ cannot be orthogonal).

### Solution

* We can find a nonzero vector by looking at the **nullspace** of the matrix $(A \ B)$.  Indeed, if $(x_1, x_2, x_3, x_4)^T$ is a nonzero vector in the nullspace of $(A \ B)$, then we have $x_1c_1 + x_2c_2 + x_3d_1 + x_4d_2$ where $c_1, c_2$ are the columns of $A$ and $d_1, d_2$ are the columns of $B$.  In particular, we then have $x_1c_1 + x_2c_2 = -(x_3d_1 + x_4d_2)$, and the lefthand side here lives in $C(A)$ and the righthand side lives in $C(B)$ and each is nonzero as $(x_1, x_2, x_3, x_4) \neq 0$ and $\{c_1, c_2\}$, $\{d_1,d_2\}$ are (separately) linearly independent.  Rather than finding such an $x$ by brute force, let's just stare carefully at the given matrices.  Notice that the first column of $B$ is three times the first column of $A$ plus the second column of $A$.  So the vector $(3, 1, -1, 0)^T$ is in the nullspace of $(A \ B)$ and corresponds to the nonzero vector $$\begin{pmatrix} 5 \\ 6 \\ 5\end{pmatrix}$$ lying in both $C(A)$ and also $C(B)$.

* We need that $V$ and $W$ are *too big to fail*.  More precisely, we need that $p + q > n$.  To see that this condition is sufficeint, find matrices $A, B$ such that $C(A) = V$ and $C(B) = W$ (this can be done by setting the columns of the matrices to be bases of the respective subspaces).  We then build the matrix $(A \ B)$ as before, and then any nonzero vector $\begin{pmatrix} x_1 \\ x_2\end{pmatrix} \in N(A \ B)$ (block form, so $x_1 \in \mathbb{R}^p, x_2 \in \mathbb{R}^q$) gives rise to a nonzero vector in the intersection $C(A) \cap C(B)$.  Such a nonzero entry exists because $(A | B)$ has $p + q$ columns and $n$ rows, and $p + q > n$, so the rank-nullity theorem guarantees it.

**Remark:** This condition is also necessary to *guarantee* a nonzero intersection, as otherwise we can write down some subspaces of dimensions $p$ and $q$ that intersect only at the origin, e.g. setting $V$ to be the subspace characterized by having all the last $n - p$ entries equal to 0, and $W$ to be the subspace characterized by having all the first $n - q$ entries equal to $0$.